In [ ]:
import flappy_bird_gym
import neat
import cv2
import numpy as np
import pickle
import gym, math
import matplotlib.pyplot as plt
import time

In [24]:
env = flappy_bird_gym.make("FlappyBird-v0")
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, 'config-feedforward.txt')
env.seed(10)
imgarray = []
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        ob = env.reset()
        net = neat.nn.recurrent.RecurrentNetwork.create(genome, config)

        current_max_fitness = 0
        fitness_current = 0
        counter = 0
        xpos = 0
        xpos_max = 0

        done = False

        while not done:

            # env.render()
            # time.sleep(.01)

            nnOutput = np.argmax(net.activate((ob[0], ob[1])))
            # ob, reward, done, info = env.step(int(round(max(nnOutput), 0)))
            ob, reward, done, info = env.step(nnOutput)


            xpos = ob[0]
            ypos = ob[1]

            if xpos < 1.66 :
                fitness_current += 1
                xpos_max = xpos

            if round(ypos, 0) == 0:
                fitness_current += 1
            else:
                fitness_current -= 1


            if reward > 1:
                fitness_current += 10
                reward += 1
            
            if done:
                fitness_current -= 10000
                # print(genome_id, fitness_current)

            if fitness_current > current_max_fitness:
                current_max_fitness = fitness_current
                counter = 0
            else:
                counter += 1
            
            if done or counter == 300:
                done = True
                # print(genome_id, fitness_current, counter)
            
            genome.fitness = fitness_current

p = neat.Population(config)

p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)
# p.add_reporter(neat.Checkpointer(10))

winner = p.run(eval_genomes)

with open('winner.pkl', 'wb') as output:
    pickle.dump(winner, output, 1)




 ****** Running generation 0 ****** 

Population's average fitness: -9916.70000 stdev: 44.88329
Best fitness: -9798.00000 - size: (2, 6) - species 1 - id 5
Average adjusted fitness: 0.187
Mean genetic distance 1.221, standard deviation 0.257
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    20  -9798.0    0.187     0
Total extinctions: 0
Generation time: 0.153 sec

 ****** Running generation 1 ****** 

Population's average fitness: -9877.80000 stdev: 59.01830
Best fitness: -9798.00000 - size: (2, 6) - species 1 - id 7
Average adjusted fitness: 0.446
Mean genetic distance 1.187, standard deviation 0.278
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    20  -9798.0    0.446     1
Total extinctions: 0
Generation time: 0.248 sec (0.201 average)

 ****** Running generation 2 ****** 

Population's average fitness: -99

In [25]:
scorelist = []
def best_model(genome, slowmode=0):
    env = flappy_bird_gym.make("FlappyBird-v0")
    env.seed(10)
    genome = pickle.load(open('winner.pkl', 'rb'))
    ob = env.reset()
    done = False
    net = neat.nn.recurrent.RecurrentNetwork.create(genome, config)

    while not done:
        
        # if slowmode == 1:
        #     time.sleep(0.01)
        #     # env.render()
        # else:
        #     # env.render()

        nnOutput = np.argmax(net.activate((ob[0], ob[1])))
        
        ob, reward, done, info = env.step(nnOutput)

    # print(info['score'])
    scorelist.append(info['score'])
    return scorelist

genome = pickle.load(open('winner.pkl', 'rb'))
for i in range(100):
    scores = (best_model(genome, 0))

print(scores)

[92, 174, 108, 385, 53, 125, 21, 35, 12, 9, 39, 23, 47, 1, 25, 211, 63, 5, 184, 65, 124, 170, 297, 10, 184, 396, 242, 18, 101, 61, 87, 138, 27, 1, 58, 16, 20, 52, 34, 6, 213, 2, 71, 203, 51, 36, 93, 11, 166, 15, 209, 17, 101, 289, 3, 13, 31, 17, 146, 11, 123, 95, 249, 36, 214, 133, 44, 3, 152, 533, 25, 100, 8, 430, 31, 73, 159, 118, 62, 171, 12, 56, 161, 46, 74, 104, 67, 11, 152, 187, 39, 6, 208, 187, 104, 104, 29, 142, 143, 278]


In [26]:
average_scores = sum(scores) / len(scores)
print(average_scores)

102.86
